In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms
import matplotlib.pyplot as plt

In [2]:
#transform an image to pytorch tensor
transform = transforms.ToTensor()
#fetch and download MNIST under './data' folder
mnisit_data = datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
data_loader = torch.utils.data.DataLoader(mnisit_data, batch_size=64,
                                          shuffle=True)

In [3]:
dataiter = iter(data_loader)
#out image has value between tensor(0.) and tensor(1.) ---> sigmoid
#if the value between tensor(-1.) and tensor(1.) ---> nn.Tanh
for images, labels in dataiter:
    print(torch.min(images), torch.max(images))

tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.) tensor(1.)
tensor(0.)

In [6]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        # N, 1, 28,28
        self.encoder = nn.Sequential(
            nn.Conv2d(1,16,3,stride=2, padding=1),#N, 16,14,14
            nn.ReLU(),
            nn.Conv2d(16,32,3,stride=2, padding=1),#N, 32,7,7
            nn.ReLU(),
            nn.Conv2d(32,64,7)#N, 64,1,1
            ) 
        #N, 64,1,1
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64,32,7),#N, 32,7,7
            nn.ReLU(),
            nn.ConvTranspose2d(32,16,3,stride=2, padding=1,output_padding=1),#N, 16,14,14
            nn.ReLU(),
            nn.ConvTranspose2d(16,1,3,stride=2, padding=1,output_padding=1),# N, 1, 28,28 
            nn.Sigmoid()
            ) 
    def forward(self,x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
#    

In [7]:
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

In [8]:
#training
num_epochs = 10
outputs = []
for epoch in range(num_epochs):
    for (img,_) in data_loader:
        #(batch_size, 28, 28) -> (batch_size, 784)
        # -1 : auto compute axis length
        #img = img.reshape(-1,784)
        recon = model(img)
        loss = criterion(recon,img)
        # Gradients accumulate after each backward pass, so resetting them to zero at the start of each iteration 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch:{epoch+1}, Loss:{loss.item():.4f}')
    outputs.append((epoch,img,recon))

Epoch:1, Loss:0.0100
Epoch:2, Loss:0.0064
Epoch:3, Loss:0.0043
Epoch:4, Loss:0.0038
Epoch:5, Loss:0.0033
Epoch:6, Loss:0.0030
Epoch:7, Loss:0.0033
Epoch:8, Loss:0.0030
Epoch:9, Loss:0.0024
Epoch:10, Loss:0.0027


In [ ]:
for k in range(0, num_epochs, 4):
    plt.figure(figsize=(9,2))
    plt.gray()
    imgs = outputs[k][1].detach().numpy()
    recon = outputs[k][2].detach().numpy()
    for i,item in enumerate(imgs):
        if i>=9: break
        plt.subplot(2,9,i+1)
        #item = item.reshape(-1,28,28)
        plt.imshow(item[0])
    for i,item in enumerate(recon):
        if i>=9: break
        plt.subplot(2,9,9+i+1) #row_length
        #item = item.reshape(-1,28,28)
        plt.imshow(item[0])